# Real-Time Crime Prediction System
This notebook builds a real-time crime prediction system with a Flask API backend and XGBoost/Random Forest models.

In [ ]:
# Install necessary packages
!pip install xgboost scikit-learn flask pyngrok pandas geopy


## Data Loading and Preprocessing

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import joblib

# Load dataset
df = pd.read_csv("/mnt/data/crime_dataset_500k.csv")

# Feature engineering
df['Hour'] = pd.to_datetime(df['Time']).dt.hour
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Day'] = pd.to_datetime(df['Date']).dt.dayofweek

# Encode categorical features
le_crime = LabelEncoder()
le_city = LabelEncoder()
df['Crime_Type'] = le_crime.fit_transform(df['Crime_Type'])
df['City'] = le_city.fit_transform(df['City'])

# Features and target
features = ['City', 'Latitude', 'Longitude', 'Hour', 'Month', 'Day']
target = 'Crime_Type'
X = df[features]
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Model Training

In [ ]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Train XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)

# Save models and encoders
joblib.dump(rf, 'random_forest_model.pkl')
joblib.dump(xgb, 'xgboost_model.pkl')
joblib.dump(le_crime, 'label_encoder_crime.pkl')
joblib.dump(le_city, 'label_encoder_city.pkl')


## Flask API with pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import joblib

# Load models
rf = joblib.load("random_forest_model.pkl")
xgb = joblib.load("xgboost_model.pkl")
le_crime = joblib.load("label_encoder_crime.pkl")
le_city = joblib.load("label_encoder_city.pkl")

app = Flask(__name__)

@app.route("/")
def home():
    return "Real-Time Crime Prediction API"

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    city = le_city.transform([data['City']])[0]
    input_data = [[
        city,
        data['Latitude'],
        data['Longitude'],
        data['Hour'],
        data['Month'],
        data['Day']
    ]]
    prediction = rf.predict(input_data)[0]
    crime = le_crime.inverse_transform([prediction])[0]
    return jsonify({"Predicted_Crime_Type": crime})

# Start Flask app with ngrok tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)
app.run()


## Frontend Setup
You can now build a React frontend to send POST requests to the `/predict` endpoint using the `public_url` shown above.